# How to Join Datasets for Modeling (and Problems to Avoid)

**In real-world ML tasks:**

- Data often comes in multiple tables.

- You need to join them to create one modeling dataset.

**If you join incorrectly:**

- You might introduce duplicate rows.

- You can cause data leakage (future info leaks into training).

- You can break the 1 row per instance assumption required by most ML algorithms.

This tutorial will teach you how to join datasets safely using pandas in Python — a key skill for any ML/Data Science project.

# Types of Joins

| Type of Join | Meaning                                                                         | Use Case                                         |
| ------------ | ------------------------------------------------------------------------------- | ------------------------------------------------ |
| INNER JOIN   | Only keep rows with matching keys in both tables                                | You only want clients with bureau history        |
| LEFT JOIN    | Keep all rows from **left table**, add data from right table (NULL if no match) | You want to keep all clients and add bureau info |
| RIGHT JOIN   | Keep all rows from **right table**, add data from left table                    | Less common in ML tasks                          |
| FULL JOIN    | Keep all rows from both tables                                                  | Rare for ML tasks                                |

In ML, we usually use:

LEFT JOIN → Safe when you want to keep all original instances.

INNER JOIN → Risky, can remove rows (sometimes useful but must be intentional).

#  Dataset Overview — Home Credit Default Risk


**Why this dataset?**

- Real-world like: multiple related tables
- Designed for tabular ML modeling → good learning case
- Main modeling table: application_train.csv
- Several "child tables": previous loans, credit cards, bureau, POS loans, etc.

**Useful for demonstrating:**

- Multi-table joining

- Aggregation before joining

- Mutating join problems

| Table                   | Primary Key                  | Comment                                              |
| ----------------------- | ---------------------------- | ---------------------------------------------------- |
| `application_train.csv` | `SK_ID_CURR` (client ID)     | Main modeling table                                  |
| `bureau.csv`            | `SK_ID_CURR`, `SK_ID_BUREAU` | Client’s bureau credit history, many rows per client |

This is a **1-to-many** relationship:

- application_train → 1 row per client
- bureau → multiple rows per client

# Mutating Join Problem Explained

In [1]:
# BAD example — naive join (just to show)
# df_bad = application_train.merge(bureau, on='SK_ID_CURR', how='left')

In this case, each client with multiple bureau records appears multiple times
- Row count of df_bad > row count of application_train
- ML model is now broken: training instances are not independent

This is called a **mutating join:**

- You change the number of rows.

- You "inflate" data incorrectly.

# Correct Pattern to Handle This

**Solution:**

Step 1:  Aggregate child table FIRST → 1 row per client

Step 2: Then LEFT JOIN the aggregate back → no row count change

# Import and Load data

In [2]:
import pandas as pd

# Load main table
application_train = pd.read_csv('application_train.csv')

# Load bureau table
bureau = pd.read_csv('bureau.csv')

print(f"Application train shape: {application_train.shape}")
print(f"Bureau shape: {bureau.shape}")


Application train shape: (307511, 122)
Bureau shape: (1716428, 17)


In [3]:
print(application_train['SK_ID_CURR'].nunique())  # number of clients
print(bureau['SK_ID_CURR'].nunique())             # number of clients in bureau table

307511
305811


# Bad Join Example -  Mutating Join


In [4]:
# Demonstrate BAD join
df_bad = application_train.merge(bureau, on='SK_ID_CURR', how='left')
print(f"BAD join shape: {df_bad.shape}")  # Will be much larger than application_train

BAD join shape: (1509345, 138)


# Correct Pattern - Aggregation before Join

In [5]:
# Aggregate bureau features per client
bureau_agg = bureau.groupby('SK_ID_CURR').agg({
    'AMT_CREDIT_SUM': ['mean', 'max'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'max'],
    'CREDIT_DAY_OVERDUE': ['max'],
    'SK_ID_BUREAU': 'count'  # number of bureau records per client
})

# Flatten columns
bureau_agg.columns = ['BUREAU_' + '_'.join(col).strip() for col in bureau_agg.columns.values]

# Reset index to prepare for join
bureau_agg = bureau_agg.reset_index()

print(f"Bureau agg shape: {bureau_agg.shape}")


Bureau agg shape: (305811, 7)


## Now perform correct join


In [6]:
# Good join: application_train + bureau_agg
df_model = application_train.merge(bureau_agg, on='SK_ID_CURR', how='left')

print(f"Final modeling dataset shape: {df_model.shape}")


Final modeling dataset shape: (307511, 128)


- Row count preserved → same as application_train
- No duplicate rows
- Suitable for modeling

# Summary & Key Takeaways

**When to Use Which Join in ML?**

- You want to keep all training instances and enrich with new features → LEFT JOIN	
- You only want instances with matching records → INNER JOIN (but be careful)	
- You need full outer info → FULL JOIN (rare)                               |                  |


**Mutating Join — Key Problem**

If you join a child table without aggregation, you will:

- Duplicate rows → breaks ML assumptions.

- Change class distribution → can cause target leakage.

Always aggregate child tables first, then join.

**Our Case:**
Chose **application_train.csv** + **bureau.csv** because it's a 1-many example → perfect to teach aggregation and safe joins.

We used **LEFT JOIN** to preserve all clients.

We demonstrated **mutating join risk** and solved it.

#  Final Notes for Your Students

When preparing data for ML:

- Think about row cardinality
- Be careful with 1-many joins
- Prefer LEFT JOIN when enriching data
- Always check row counts before and after join → sanity check
- Aggregate child tables first → this is a safe and common pattern!

In [7]:
# Sanity Check
print("Row count before join:", application_train.shape[0])
print("Row count after safe join:", df_model.shape[0])

Row count before join: 307511
Row count after safe join: 307511


 If both match → you did it right!

# Why This Matters in Real ML Work

In Machine Learning:

- 1-many relationships are everywhere: transaction history, logs, clickstream, credit history, etc.

- Safe joining is an essential skill → prevents data leakage and model corruption.

- If you blindly join tables → you'll get bad models and possibly wrong business decisions.